In [249]:
# If you what use ngrams.py you should use python2
# Or, otherwise, you need to modify ngrams.py by yourself in order to use it in python3.
from __future__ import print_function
from __future__ import unicode_literals
from string import punctuation
import re
import numpy as np
from ngrams import ngrams
from collections import defaultdict
from bitweight import BitWeight, BitWeightRangeError

In [250]:

small_corpus = ['Why dont we start here',
                  'Why dont we end there',
                  'Let us start with a few other examples',
                  'We never start with an example with so few tokens',
                  'Tokens can be words that we start with in example docs']


In [251]:
def tokenize(corpus):
    tokens = [sentence.split(' ') for sentence in corpus]
    return tokens


# HW3: Language Modeling
For this part of the assignment, you will implement two simple count-based n-gram language models: one based on maximum-likelihood estimation, and another based on Witten-Bell smoothing. The data you will be using is a subset of the Penn Treebank's tagged Wall Street Journal articles on which we have done some initial processing. There are two versions of the data for this assignment:

##### wsj.pos.gz
##### wsj-normalized.pos.gz
The difference is that, in the second (normalized) version of the data, we have collapsed some entries from certain tag categories (e.g. CDs, NNPs, etc.) into type-tokens to help reduce sparsity. Take a look at the data and see for yourself. Consider: what would be the benefits and drawbacks to this method of sparsity reduction? Note that, for this part of the assignment, the tags are un-necessary, so you'll want to work with the un-normalized version of the corpus.

### Task 1: produce a tag-free corpus

For this task, you have two jobs. 
* First, you need to write a function to filter out all tags. 
* Second, Make sure your code works for both wsj.pos.gz and wsj-normalized.pos.gz

####What to turn in
* your code
* some samples to show me that your code works as it should be

In [252]:
def file_to_list(filename):
    with open(filename, 'r') as content_file:
        content = content_file.read()
        no_tags = re.sub('(/[A-Z$]{2,4})|(\.\s+[a-z])|(/[.,$])|[\\,/\'`]', '', content)
        return re.split('[\n]',no_tags)
        
s = file_to_list("./wsj.pos")
#for x in range(10): print(s[x],'\n')

<font color="red">Self assessment:</font>

### Maximum Likelihood
Now, start by producing code to compute maximum-likelihood estimate probabilities. Your code should be configurable with respect to the n-gram order- i.e., you should be able to set it to compute bigram, trigram, 4-gram, etc. probabilities. Refer to J&M and the lecture slides for definitions as needed. If you would like to write your own n-gram tokenization code, feel free to do so, but you may also use the ngrams.py utility class which contains a routine to take a list of tokens and produce a stream of n-grams with appropriate padding for the start and end of sentences.

#### Tip: 
* Start with a very small "toy" corpus of just a couple of sentences for debugging. 

* As discussed in class, I strongly recommend using nested defaultdicts as the foundational data structure for your language model, where the "outer" key is the prefix, and the value retrieved by that prefix is a second defaultdict  containing possible suffices for that prefix, each of which is an "inner" key. E.g., p("TRUTHS" | "HOLD THESE") would be retrieved by first looking up "HOLD THESE" and then from the resulting dictionary, looking up "TRUTHS": prob = trigrams[("HOLD","THESE")]["TRUTHS"] . Note that this arrangement makes it very easy to e.g. find out the number of times a given history occurs, the total probability mass assigned to all of a history's continuations, etc., all of which will be extremely helpful in the next part of the assignment.

* Use tuples to represent prefixes. E.g., instead of the string "HOLD THESE", use the tuple ("HOLD", "THESE"). Note that, in Python, lists are mutable, and therefore may not be used as keys in dictionaries- but tuples are immutable, and so make excellent keys.

* Don't forget about numerical underflow issues! You'll want to represent probabilities as negative base-2 log probabilities, and modify your arithmetic accordingly. I recommend experimenting with [the bitweight Python library](https://github.com/stevenbedrick/bitweight) (see its unit tests for example usage).
* 

#### What to turn in:
* your code 
* use your code to create a simple language model for small_corpus named as small_lm and show me that your output is correct(This is a small coupus so you could manully calculate the probalility).
* use your code to create language model for wsj.pos.gz named as wsj_lm

In [253]:

# generates a model with just counts
def count_builder(corpus, order):
    
    #ngram
    ng = ngrams(corpus, order)

    # describe model datatype
    model = defaultdict(lambda: defaultdict(int))
    
    # loop to build embedded defaultdict    
    for gram in ng: 
        if not gram[1] in model[gram[0]]:
            model[gram[0]][gram[1]] = 1
        else:
            model[gram[0]][gram[1]] += 1

    return model



# Takes an input of a count model and outputs a MLE model
def max_likelihood(count_model):
    
    #returned MLE Model with BitWeight Probabilities
    model = defaultdict(lambda: defaultdict(lambda: BitWeight(0)))
    
    # for prefixes in model
    for prefix, suffix_dict in count_model.iteritems():
        w_minus = BitWeight(0)
        # for words with hist prefix
        for suffix, count in suffix_dict.iteritems():
            w_minus += BitWeight(count) # count number of tokens
        for suffix, count in suffix_dict.iteritems():
            model[prefix][suffix] = BitWeight.__itruediv__(BitWeight(count),w_minus) # set output probabilities
    
    return model



# model printer
def model_printer(model):
    total = BitWeight(0)
    count = 0 
    for prefix, suffix_dict in model.iteritems() :
        for suffix, value in suffix_dict.iteritems():
            print(prefix," : ", suffix, " : ", value.real())
            count += 1
            total += value
    print("Total Probability: ",total.real())
    print("Count: ",count)
            

small_tokens = tokenize(small_corpus) # turn small corpus into ngrams
count_model = count_builder(small_tokens, 2) # build bigram count model
ml_model = max_likelihood(count_model) # build max likelihood model from count model
model_printer(ml_model) # print entire model

x=ngrams(small_tokens,2)
for i in ml_model:
    print(i)
ml_model[('<S_0>',)]['Why'].real()

(u'examples',)  :  </S_0>  :  1.0
(u'few',)  :  tokens  :  0.5
(u'few',)  :  other  :  0.5
(u'in',)  :  example  :  1.0
(u'We',)  :  never  :  1.0
(u'Why',)  :  dont  :  1.0
(u'end',)  :  there  :  1.0
(u'start',)  :  with  :  0.75
(u'start',)  :  here  :  0.25
(u'other',)  :  examples  :  1.0
(u'here',)  :  </S_0>  :  1.0
(u'words',)  :  that  :  1.0
(u'an',)  :  example  :  1.0
(u'we',)  :  start  :  0.666666666667
(u'we',)  :  end  :  0.333333333333
(u'dont',)  :  we  :  1.0
(u'there',)  :  </S_0>  :  1.0
('<S_0>',)  :  Tokens  :  0.2
('<S_0>',)  :  We  :  0.2
('<S_0>',)  :  Let  :  0.2
('<S_0>',)  :  Why  :  0.4
(u'so',)  :  few  :  1.0
(u'us',)  :  start  :  1.0
(u'a',)  :  few  :  1.0
(u'example',)  :  docs  :  0.5
(u'example',)  :  with  :  0.5
(u'docs',)  :  </S_0>  :  1.0
(u'Tokens',)  :  can  :  1.0
(u'never',)  :  start  :  1.0
(u'Let',)  :  us  :  1.0
(u'can',)  :  be  :  1.0
(u'be',)  :  words  :  1.0
(u'with',)  :  a  :  0.25
(u'with',)  :  an  :  0.25
(u'with',)  :  so  

0.3999999999999999

<font color="red">Self assessment:</font>

### Smoothing

Once you’ve got an unsmoothed model working, move on to implementing Witten-Bell smoothing. Refer to the slides and J&M for details on how that ought to work.

#### Tip: 
* You can modify an already-populated defaultdict to change its default value (for example, to store a default backoff value for a particular history) by changing the object’s default_factory attribute. Consult the documentation for examples of how this works.
* As defined, W-B smoothing is highly recursive; you may find it more efficient to re-frame the algorithm in iterative terms.
* As in the previous section, start small.
* [This may offer you some help on how to implement Witten-Bell smoothing](http://www.ee.columbia.edu/~stanchen/e6884/labs/lab3/x207.html)


#### What to turn in:
* your code 
* use your code to create a simple smoothed language model based on small_lm  and show me that your output is correct(This is a small coupus so you could manully calculate the probalility).
* use your code to create a smoothed language model based on wsj_lm

<font color="red">Self assessment:</font>

### Evaluation via Perplexity
Explore the effects of n-gram order using perplexity. Perform ten-fold cross-validation on the WSJ corpus. On each iteration, this will give you a different 90/10 training/test split; train a smoothed language model on the 9 training sections, and compute the average per-token perplexity of the tenth section. The slides from the language modeling lecture give the equation for perplexity computation (as does J&M chapter 4); you'll need to modify the equation a bit, since we're using log-probabilities. 

Now, try this for unigram, bigram, trigram, and 4-gram models. 

#### What to turn in
* your cross-validation function. You are not suppose to use any cross-validation function from any module. You should implement it by yourself.
* your perplexity function
* cross-validation result for unigram, bigram, trigram, and 4-gram models on wsj.pos.gz
* cross-validation result for unigram, bigram, trigram, and 4-gram models on wsj-normalized.pos.gz.
* Answer following 2 questions: 
    * How does perplexity change as the model order size increases?
    * How does perplexity change as the data changed?

In [254]:

def wb_model_builder(tokens, order):
    
    # build count model and MLE model to calculate WB values
    count_mod = count_builder(tokens, order)
    mle_mod = max_likelihood(count_mod)
    
    
    # Base Case: Unigram Model Builder
    if order == 1:
        
        # get tokens and types to calc lmbda(.)
        tks = sum(count_mod[()].values())
        tps = len(count_mod[()].keys())
        
        # use above to build lambda(.)
        lambda_dot = BitWeight(tks,(tks+tps))
        one_minus_lambda_dot = BitWeight(tps,(tks+tps))
        one_over_V = BitWeight(1,tps)
        
        # create return for unseen values Pwb(.) = (1 - lambda) * (1/V)
        default_return = one_minus_lambda_dot*one_over_V
        
        # declare WB unigram model to be handed back
        wb_model = defaultdict(lambda: defaultdict(lambda: default_return))
        
        # build WB unigram model
        for prefix, suffix_dict in mle_mod.iteritems() :
            for suffix, mle_prob in suffix_dict.iteritems():
                # Set WB Probability: Pwb(w) = lambda(.)*Pmle + one_minus_lambda(.)*one_over_V
                wb_model[prefix][suffix] = (lambda_dot * mle_prob) + (one_minus_lambda_dot*one_over_V)
        
        # return WB unigram model
        return wb_model
    
    
    # Recursive Case: If the order is greater than 1
    
    # get the model of P(w|h_n-1)
    wb_minus_one_model = wb_model_builder(tokens, order-1)
    
    # retrieve default value from lower order model
    default_return = wb_minus_one_model["x"]["x"] # xx will not be in any model

    # declare WB model with inhereted default
    wb_model = defaultdict(lambda: defaultdict(lambda: default_return))
    
    # for each history... 
    for prefix, suffix_dict in count_mod.iteritems():

        # calculate history parameters
        c_hist = sum(suffix_dict.values())
        N_one_plus = len(suffix_dict.keys())

        # build lambda terms
        lambda_hist = BitWeight(c_hist,(c_hist+N_one_plus))
        one_minus_lambda_hist = BitWeight(N_one_plus,(c_hist+N_one_plus))
        
        # for each word that follows that history ... 
        for suffix, wb_minus_prob in mle_mod[prefix].iteritems():

            # Set WB Probability: Pwb(w|hist) = lambda_hist*Pmle(w|hist) + one_minus_lambda_hist*Pwb(w|hist-1)
            lambda_hist_x_mle = lambda_hist*mle_mod[prefix][suffix]
            one_minus_lambda_x_wb_minus = one_minus_lambda_hist*wb_minus_one_model[prefix[1:]][suffix]
            wb_model[prefix][suffix] = lambda_hist_x_mle + one_minus_lambda_x_wb_minus
    
    # return WB ngram model
    return wb_model


In [257]:
#sx = tokenize(x)

m = wb_model_builder(small_tokens, 6)
model_printer(m)


('<S_2>', '<S_3>', '<S_4>', u'Tokens', u'can')  :  be  :  0.507575757576
(u'we', u'start', u'with', u'in', u'example')  :  docs  :  0.954071969697
('<S_0>', '<S_1>', '<S_2>', '<S_3>', '<S_4>')  :  Tokens  :  0.117845117845
('<S_0>', '<S_1>', '<S_2>', '<S_3>', '<S_4>')  :  We  :  0.117845117845
('<S_0>', '<S_1>', '<S_2>', '<S_3>', '<S_4>')  :  Let  :  0.117845117845
('<S_0>', '<S_1>', '<S_2>', '<S_3>', '<S_4>')  :  Why  :  0.228956228956
('<S_4>', u'We', u'never', u'start', u'with')  :  an  :  0.90459280303
(u'with', u'a', u'few', u'other', u'examples')  :  </S_4>  :  0.507575757576
(u'Why', u'dont', u'we', u'start', u'here')  :  </S_4>  :  0.507575757576
(u'start', u'here', '</S_4>', '</S_3>', '</S_2>')  :  </S_1>  :  0.507575757576
('<S_2>', '<S_3>', '<S_4>', u'Let', u'us')  :  start  :  0.507575757576
(u'here', '</S_4>', '</S_3>', '</S_2>', '</S_1>')  :  </S_0>  :  0.507575757576
('<S_4>', u'Why', u'dont', u'we', u'start')  :  here  :  0.917297979798
('<S_2>', '<S_3>', '<S_4>', u'Why

<font color="red">Self assessment:</font>

In [89]:
tup = ('a','b','c')

In [92]:
tup = tup[1:]
tup

()